In [8]:
import sys
sys.path.append('../')

from src.filters.f_cytotox import CytotoxicityFilter
from src.filters.f_distribution import DistributionFilter
from src.utils.gen_esm_embedds import Embedder

from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
INPUT_FOLDER = Path('../src/data/mut_DBs/progen-acp-mp')

CTT_PATH = Path('../src/models/classificators/RFC_esm.pkl')
DIST_DATA_PATH = Path('../src/data/acp_trainset.csv')
OG = pd.read_csv('../src/data/known_acps/acp_mp.csv',index_col=False)

In [3]:
filters = []
filters.append(CytotoxicityFilter(CTT_PATH, 'mps'))
filters.append(DistributionFilter(DIST_DATA_PATH, '../src/charge.json','../src/hydrophobicity.json'))

/Users/macbook/miniconda3/envs/acp_gen/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/macbook/miniconda3/envs/acp_gen/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


initializing the embedding model


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
### remove already existing acps ###
DB = pd.read_csv(INPUT_FOLDER / 'DB_raw.csv',index_col=False)
DB_filtered = DB[~DB['sequence'].isin(set(OG['sequence']))]
DB_filtered.to_csv(INPUT_FOLDER / 'DB_filtered.csv', index=False)

DB_filtered

,sequence,label
0,IKDWAEKVMQAL,GEN
1,ILVDKPFYRALQLSKDDILAL,GEN
2,IKLNMAGIKCRGLCGLLPLAI,GEN
3,KLWIRAPLERVNNKLM,GEN
4,IKDLLAFLSIDTYNLPRLIGI,GEN
...,...,...
355880,ILMWKALAAT,GEN
355881,LINWKGLASIVISAL,GEN
355882,INLDAVASRMEIAS,GEN
355883,IKLDGSPMWYRDAALFSRL,GEN


In [5]:
for filter in filters:
    DB_filtered = filter.filter_sequences(DB_filtered)

DB_filtered.to_csv(INPUT_FOLDER / 'DB.csv',index=False)

generating sequence embeddings


100%|██████████| 5561/5561 [03:48<00:00, 24.31it/s]
/Users/macbook/Documents/RoVa/Masters/ACP_designer/ACP_designer_WEB_repo/ACP-designer-WEB/notebooks/../src/filters/f_cytotox.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['toxicity_prob'] = probabilities
/Users/macbook/Documents/RoVa/Masters/ACP_designer/ACP_designer_WEB_repo/ACP-designer-WEB/notebooks/../src/filters/f_cytotox.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['toxicity_cat'] = np.select(
/Users/macbook/Documents/RoVa/Master

In [6]:
sequences = list(DB_filtered['sequence'].to_numpy())

embedder = Embedder('mps')
embeddings = embedder.get_embeddings(sequences)
np.save(INPUT_FOLDER / 'esm_embedds_filtered.npy',embeddings)

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 5561/5561 [03:47<00:00, 24.41it/s]


**just embedding**

In [12]:
acps_db = pd.read_csv('/Users/macbook/Documents/RoVa/Masters/ACP_designer/ACP_designer_WEB_repo/ACP-designer-WEB/src/data/acps/ACP.csv',index_col=False)
acp_sequences = list(acps_db['sequence'].to_numpy())

mps_db = pd.read_csv('/Users/macbook/Documents/RoVa/Masters/ACP_designer/ACP_designer_WEB_repo/ACP-designer-WEB/src/data/mps/MP.csv',index_col=False)
mp_sequences = list(mps_db['sequence'].to_numpy())

embedder = Embedder('mps')
embeddings_acp = embedder.get_embeddings(acp_sequences)
embeddings_mp = embedder.get_embeddings(mp_sequences)
np.save( '/Users/macbook/Documents/RoVa/Masters/ACP_designer/ACP_designer_WEB_repo/ACP-designer-WEB/src/data/acps/esm_embedds_acp.npy',embeddings_acp)
np.save( '/Users/macbook/Documents/RoVa/Masters/ACP_designer/ACP_designer_WEB_repo/ACP-designer-WEB/src/data/mps/esm_embedds_mp.npy',embeddings_mp)

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 40/40 [00:02<00:00, 15.06it/s]
